In [1]:
import pandas as pd, numpy as np

In [2]:
pd.options.display.max_columns = 99
from collections import Counter
import re
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [194]:
from sklearn.externals import joblib

In [149]:
from scipy.stats import gmean
from datetime import datetime, timedelta

In [4]:
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
# from sklearn.linear_model import Au
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

/home/lolik111/anaconda3/envs/cud/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import pandas_profiling

In [6]:
from collections import Iterable

class LabelCountEncoder(object):
    """
    Class to encode labels from most to least frequent
    """
    
    def __init__(self, most_freq_labels=None):
        if issubclass(type(most_freq_labels), Iterable) and not issubclass(type(most_freq_labels), str):
            self.mfl = list(most_freq_labels)
        else:
            self.mfl = [most_freq_labels]
        
    def fit_transform(self, X):
        lbls, inv, cnts = np.unique(X, return_inverse=True, return_counts=True)
        inf = np.iinfo(np.int32).max
        cnts[np.isin(lbls, self.mfl)] = inf
        self.sorted_idx = np.argsort(-cnts)
        self.inv_sorted_idx = np.argsort(self.sorted_idx)
        self.invt = lbls[self.sorted_idx]
        return self.inv_sorted_idx[inv]
    
    def inverse_transform(self, y):
        return self.invt[y]

In [7]:
train = pd.read_csv("./data/train.csv", parse_dates=[0, 5, 6, 7, 8]) 

In [8]:
test = pd.read_csv("./data/test.csv", parse_dates=[0, 5, 6])

In [185]:
additional_data = pd.read_csv("./data/flight_leg_exp_joined_with_metar.csv", parse_dates=["SCH_START_DATE_TIME",
    "SCH_END_DATE_TIME", "ORIGINAL_FLIGHT_DATE", "FLIGHT_DATE"])

/home/lolik111/anaconda3/envs/cud/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
additional_data.DISPLAY_CODE.unique()

array(['SVOA320SU', 'SVOA321SU', 'SVOA333SU', 'SVOSU95SU', 'SVOA319SU',
       'SVOA32RSU', 'SVOA3KRSU', 'SVOIL96SU', 'SVOB763SU', 'SVOA332SU',
       'SVOB777SU', 'SVOB737SU', 'SVOB73HSU', 'SVOA32ASU', 'SVOA32BSU',
       'SVOA31USU'], dtype=object)

In [78]:
additional_data[additional_data.DISPLAY_CODE == "SVOB777SU"].TAIL_NUMBER.unique()

array(['VPBGB', 'VPBGC', 'VPBGD', 'VPBGF', 'VQBQB', 'VQBQC', 'VQBQD',
       'VQBQE', 'VQBQF', 'VQBQG', 'VQBUA', 'VQBUB', 'VQBUC', 'VQBQM',
       'VQBIL', 'VPBPG'], dtype=object)

In [188]:
additional_data.ORIGINAL_FLIGHT_DATE.dt.time.unique()

array([datetime.time(20, 0), datetime.time(21, 0)], dtype=object)

In [211]:
toj = additional_data.set_index(["SCH_START_DATE_TIME", "SCH_END_DATE_TIME", "DEPARTURE_AIRPORT_CODE", "ARRIVAL_AIRPORT_CODE"])

In [219]:
lj = train.copy()

In [226]:
lj["Время отправления по расписанию"] = \
        train["Время отправления по расписанию"] - timedelta(hours=6)
lj["Время прибытия по расписанию"] = \
        train["Время прибытия по расписанию"] - timedelta(hours=6)

In [231]:
merged = lj.join(toj, on=["Время отправления по расписанию", "Время прибытия по расписанию", "А/П отправл", "А/П прибыт"],
                how="inner")

In [259]:
planes = list(set(test["Номер ВС"].unique()) - set(merged["Номер ВС"].unique()))

In [264]:
flights = list(set(test["Рейс"].unique()) - set(merged["Рейс"].unique()))

In [414]:
planes.__len__(), flights.__len__()

(59, 212)

In [413]:
test.nunique()

Дата рейса                            83
Рейс                                1064
А/П отправл                          164
А/П прибыт                           167
Номер ВС                             246
Время отправления по расписанию    22013
Время прибытия по расписанию       22516
dtype: int64

# Dataset processing

In [9]:
train = train.append(test, ignore_index=True, sort=False)

In [10]:
train

,Дата рейса,Рейс,А/П отправл,А/П прибыт,Номер ВС,Время отправления по расписанию,Время отправления фактическое,Время прибытия по расписанию,Время прибытия фактическое,1 КЗ Код,Задержка отправления в минутах
0,2015-10-27,387,SVO,HAV,127,2015-10-27 07:40:00,2015-10-27 07:36:00,2015-10-27 20:45:00,2015-10-27 20:57:00,NaN,0.0
1,2015-10-27,1,SVO,JFK,235,2015-10-27 09:50:00,2015-10-27 09:52:00,2015-10-27 20:35:00,2015-10-27 20:39:00,03.608,2.0
2,2015-10-27,37,SVO,MIA,194,2015-10-27 10:45:00,2015-10-27 10:41:00,2015-10-27 23:35:00,2015-10-27 23:23:00,NaN,0.0
3,2015-10-27,29,SVO,LAX,196,2015-10-27 12:30:00,2015-10-27 12:24:00,2015-10-28 01:20:00,2015-10-28 01:07:00,NaN,0.0
4,2015-10-27,671,OTP,SVO,18,2015-10-27 14:15:00,2015-10-27 14:24:00,2015-10-27 16:40:00,2015-10-27 17:10:00,55,9.0
5,2015-10-27,845,HAM,SVO,164,2015-10-27 14:30:00,2015-10-27 14:30:00,2015-10-27 17:15:00,2015-10-27 17:09:00,93,0.0
6,2015-10-27,15,SVO,JFK,241,2015-10-27 14:35:00,2015-10-27 14:30:00,2015-10-28 01:25:00,2015-10-28 01:08:00,NaN,0.0
7,2015-10-27,1239,DXB,SVO,255,2015-10-27 15:40:00,2015-10-27 15:41:00,2015-10-27 21:20:00,2015-10-27 20:57:00,03.608,1.0
8,2015-10-27,478,SVO,VVO,203,2015-10-27 16:10:00,2015-10-27 16:04:00,2015-10-28 00:35:00,2015-10-28 00:34:00,NaN,0.0
9,2015-10-27,1227,TLV,SVO,200,2015-10-27 16:45:00,2015-10-27 16:41:00,2015-10-27 20:55:00,2015-10-27 20:44:00,NaN,0.0


In [11]:
# train
target = train["Задержка отправления в минутах"]
train = train.drop(["Дата рейса", "Время отправления фактическое", "Время прибытия фактическое", #"1 КЗ Код", 
                    "Задержка отправления в минутах"], axis=1)

In [12]:
col = "Время отправления по расписанию"
train[col + "_year"] = train[col].dt.year
train[col + "_month"] = train[col].dt.month
train[col + "_day"] = train[col].dt.day
train[col + "_hour"] = train[col].dt.hour
train[col + "_minute"] = train[col].dt.minute
train[col + "_dow"] = train[col].dt.dayofweek
train[col + "_epoch"] = train[col].astype(np.int64) // 1000000000

col = "Время прибытия по расписанию"
train[col + "_year"] = train[col].dt.year
train[col + "_month"] = train[col].dt.month
train[col + "_day"] = train[col].dt.day
train[col + "_hour"] = train[col].dt.hour
train[col + "_minute"] = train[col].dt.minute
train[col + "_dow"] = train[col].dt.dayofweek
train[col + "_epoch"] = train[col].astype(np.int64) // 1000000000

In [13]:
train["arrival_diff"] = (train["Время прибытия по расписанию"] - train["Время отправления по расписанию"]).dt.seconds

In [36]:
def parse_xml(root):
    
    keys = ["url", "price", "currencyId", "categoryId", "picture", "vendor", "vendorCode", "model"]
    
    result = {}
    for airport in root.getchildren():
        res_airp = {}
        res_airp["type"] = airport.attrib['type']
        res_airp["season"] = airport.attrib['season']
        res_airp["town"] = airport.find("town").attrib['town']
        res_airp["country"] = airport.find("town").attrib['country']
        res_airp["latitude"] = airport.find("coordinates").find('latitude').attrib['value']
        res_airp["longitude"] = airport.find("coordinates").find('longitude').attrib['value']
        if 'value' in airport.find("size").attrib:
            res_airp["size_value"] = airport.find("size").attrib['value']
        else: 
            res_airp["size_value"] = "Medium"
        
        if 'pax' in airport.find("size").attrib:
            res_airp["size_pax"] = int(airport.find("size").attrib['pax'])
        else:
            res_airp["size_pax"] = np.NaN
            
        terminals = airport.find("terminals").getchildren()
        
        res_airp["num_terminals"] = terminals.__len__()
        res_airp["num_gates"] = sum([int(t.attrib['gates']) for t in terminals])
        
        runways = airport.find("runways").getchildren()
        res_airp["num_runways"] = runways.__len__()
        c = Counter([r.attrib['surface'] for r in runways])
        if (c.__len__() > 0):
            res_airp["runway_material"] = c.most_common(1)[0][0]
        else:
            res_airp["runway_material"] = np.NaN
        res_airp["runway_len"] = np.mean([int(r.attrib['length']) for r in runways])
        result[airport.attrib['iata']] = res_airp
    return result

In [23]:
def parse_lat(latitude):
    N = 'N' in latitude
    ch_arr = [str(i) for i in range(10)]
    while(latitude[-1] not in ch_arr):
        latitude = latitude[:-1]
        
    rms = lambda x: re.sub("\D", '', x)
    
    d = float(rms(latitude.split('°')[0]))
    m, s = latitude.split('°')[1].split("''")[0].split("'")
    m, s = float(rms(m)), float(rms(s))
    ret = (d + m / 60. + s / 3600.) * (1 if N else -1)
    return ret

def parse_lon(longitude):
    W = 'W' in longitude
    ch_arr = [str(i) for i in range(10)]
    while(longitude[-1] not in ch_arr):
        longitude = longitude[:-1]
    
    rms = lambda x: re.sub("\D", '', x)
    d = float(rms(longitude.split('°')[0]))
    m, s = longitude.split('°')[1].split("''")[0].split("'")
    
    m, s = float(rms(m)), float(rms(s))
    ret = (d + m / 60. + s / 3600.) * (1 if W else -1)
    return ret

In [ ]:
df_airport = pd.DataFrame(airport_data).transpose()
df_airport.loc[df_airport.size_pax.isna(), "size_pax"] = 960
df_airport.size_pax = df_airport.size_pax.astype(np.int32)
df_airport.loc[df_airport.runway_len.isna(), "runway_len"] = 0
df_airport.loc[df_airport.runway_len.isna(), "runway_material"] = "Asphalt"
df_airport.loc[:, ["num_gates", "num_runways", "num_terminals", "runway_len"]] = \
    df_airport[["num_gates", "num_runways", "num_terminals", "runway_len"]].astype(np.int32)
df_airport.loc[df_airport.longitude == "109'57'48''E", "longitude"] = "109°57'48''E"

In [24]:
df_airport.loc[:, "latitude"] = df_airport.latitude.apply(parse_lat)
df_airport.loc[:, "longitude"] = df_airport.longitude.apply(parse_lon)

In [25]:
mtrain = train.join(df_airport, on="А/П отправл", rsuffix="departure").join(df_airport, on="А/П прибыт", rsuffix="_arrival")

In [27]:
cols = ["Рейс", "А/П отправл", "А/П прибыт", "Номер ВС"]
lces = dict(zip(cols, [LabelCountEncoder() for i in range(len(cols))]))
# LabelCountEncoder().fit_transform(mtrain["Рейс"])

In [28]:
for k, v in lces.items():
    mtrain[k + "_lce"] = lces[k].fit_transform(mtrain[k])

In [35]:
mtrain

,Рейс,А/П отправл,А/П прибыт,Номер ВС,Время отправления по расписанию,Время прибытия по расписанию,1 КЗ Код,Время отправления по расписанию_year,Время отправления по расписанию_month,Время отправления по расписанию_day,Время отправления по расписанию_hour,Время отправления по расписанию_minute,Время отправления по расписанию_dow,Время отправления по расписанию_epoch,Время прибытия по расписанию_year,Время прибытия по расписанию_month,Время прибытия по расписанию_day,Время прибытия по расписанию_hour,Время прибытия по расписанию_minute,Время прибытия по расписанию_dow,Время прибытия по расписанию_epoch,arrival_diff,country,latitude,longitude,num_gates,num_runways,num_terminals,runway_len,runway_material,season,size_pax,size_value,town,type,country_arrival,latitude_arrival,longitude_arrival,num_gates_arrival,num_runways_arrival,num_terminals_arrival,runway_len_arrival,runway_material_arrival,season_arrival,size_pax_arrival,size_value_arrival,town_arrival,type_arrival,Рейс_lce,А/П отправл_lce,А/П прибыт_lce,Номер ВС_lce
0,387,SVO,HAV,127,2015-10-27 07:40:00,2015-10-27 20:45:00,NaN,2015,10,27,7,40,1,1445931600,2015,10,27,20,45,1,1445978700,47100,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,198,23.016667,82.383056,17,1,1,4000,Asphalt,AllYear,4119,Medium,Havana,LongHaulInternational,452,0,109,196
1,1,SVO,JFK,235,2015-10-27 09:50:00,2015-10-27 20:35:00,03.608,2015,10,27,9,50,1,1445939400,2015,10,27,20,35,1,1445978100,38700,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,122,40.639722,73.778889,137,4,7,3372,Asphalt,AllYear,50423,Verylarge,"New York, NY",LongHaulInternational,134,0,46,174
2,37,SVO,MIA,194,2015-10-27 10:45:00,2015-10-27 23:35:00,NaN,2015,10,27,10,45,1,1445942700,2015,10,27,23,35,1,1445988900,46200,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,122,25.793056,80.290556,120,4,3,3159,Asphalt,Summer,40500,Verylarge,"Miami, FL",LongHaulInternational,669,0,124,187
3,29,SVO,LAX,196,2015-10-27 12:30:00,2015-10-28 01:20:00,NaN,2015,10,27,12,30,1,1445949000,2015,10,28,1,20,2,1445995200,46200,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,122,33.942500,118.408056,112,4,9,3230,Concrete,AllYear,66667,Largest,"Los Angeles, CA",LongHaulInternational,101,0,101,190
4,671,OTP,SVO,18,2015-10-27 14:15:00,2015-10-27 16:40:00,55,2015,10,27,14,15,1,1445955300,2015,10,27,16,40,1,1445964000,8700,147,44.573611,-26.103333,24,2,1,3500,Asphalt,AllYear,7643,Medium,Bucharest,LongHaulInternational,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,215,90,0,122
5,845,HAM,SVO,164,2015-10-27 14:30:00,2015-10-27 17:15:00,93,2015,10,27,14,30,1,1445956200,2015,10,27,17,15,1,1445966100,9900,1002,53.630278,-9.988056,25,2,1,3458,Asphalt,AllYear,13558,Large,Hamburg,LongHaulInternational,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,18,84,0,53
6,15,SVO,JFK,241,2015-10-27 14:35:00,2015-10-28 01:25:00,NaN,2015,10,27,14,35,1,1445956500,2015,10,28,1,25,2,1445995500,39000,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,122,40.639722,73.778889,137,4,7,3372,Asphalt,AllYear,50423,Verylarge,"New York, NY",LongHaulInternational,222,0,46,185
7,1239,DXB,SVO,255,2015-10-27 15:40:00,2015-10-27 21:20:00,03.608,2015,10,27,15,40,1,1445960400,2015,10,27,21,20,1,1445980800,20400,156,25.254722,-55.364167,101,2,2,4223,Asphalt,AllYear,66431,Largest,Dubai,LongHaulInternational,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,541,73,0,160
8,478,SVO,VVO,203,2015-10-27 16:10:00,2015-10-28 00:35:00,NaN,2015,10,27,16,10,1,1445962200,2015,10,28,0,35,2,1445992500,30300,149,55.971667,-37.415000,28,2,1,3625,Concrete,AllYear,29256,Large,Moscow,LongHaulInternational,149,43.399167,-132.151389,16,4,1,1893,Asphalt,AllYear,1853,

In [29]:
cat_train = mtrain.drop(["Время отправления по расписанию", "Время прибытия по расписанию", "1 КЗ Код"], axis=1)

In [38]:
mtrain.to_csv("./parsed_data.csv", sep=';')

In [43]:
train.shape

(740943, 22)

In [41]:
target.to_csv("./target.csv")

In [43]:
train.shape[0] - test.shape[0]

675513

# Catboost

In [65]:
from catboost import CatBoostRegressor, Pool

In [228]:
cat_params = dict(bootstrap_type='Bernoulli', one_hot_max_size=8, loss_function='RMSE',
                     max_depth=8, learning_rate=0.1, n_estimators=2000, reg_lambda=.5, verbose=True,
                  thread_count=6, subsample=1., rsm=1., use_best_model=True, task_type='GPU')
#, max_ctr_complexity=6, ctr_border_count=32)
#task_type='GPU', #bagging_temperature=.25, #use_best_model=True,
                      #random_strength=1., gpu_cat_features_storage='GpuRam',# pinned_memory_size=2 *1073741824) #

In [41]:
cbr = CatBoostRegressor(**cat_params)

In [55]:
lx = train.shape[0] - test.shape[0]
X, y = cat_train.values[:lx], target.values[:lx]

In [54]:
icats = [i for i, dtt in enumerate(cat_train.dtypes) if not np.issubdtype(dtt, np.number)]

In [56]:
idx_tr, idx_te = next(ShuffleSplit(test_size=0.2, random_state=1234).split(X, y))

In [196]:
np.sort(caticat)

array([ 0,  1,  2,  3,  9, 16, 19, 26, 27, 29, 30, 31, 32, 39, 40, 42, 43,
       44, 45, 46, 47, 48])

In [229]:
cbr = CatBoostRegressor(**cat_params)
cbr.fit(X[idx_tr], y[idx_tr], cat_features=np.sort(caticat), eval_set=(X[idx_te], y[idx_te]))

0:	learn: 44.6765139	test: 45.9584983	best: 45.9584983 (0)	total: 169ms	remaining: 5m 37s
1:	learn: 43.8135949	test: 45.0215521	best: 45.0215521 (1)	total: 333ms	remaining: 5m 32s
2:	learn: 43.0796993	test: 44.2216721	best: 44.2216721 (2)	total: 485ms	remaining: 5m 22s
3:	learn: 42.4723363	test: 43.5525521	best: 43.5525521 (3)	total: 619ms	remaining: 5m 8s
4:	learn: 41.9995522	test: 43.0146269	best: 43.0146269 (4)	total: 729ms	remaining: 4m 51s
5:	learn: 41.5751049	test: 42.5471040	best: 42.5471040 (5)	total: 882ms	remaining: 4m 53s
6:	learn: 41.2143983	test: 42.1561143	best: 42.1561143 (6)	total: 1.02s	remaining: 4m 50s
7:	learn: 40.8899167	test: 41.8321228	best: 41.8321228 (7)	total: 1.17s	remaining: 4m 52s
8:	learn: 40.6301628	test: 41.5271920	best: 41.5271920 (8)	total: 1.31s	remaining: 4m 50s
9:	learn: 40.3989021	test: 41.2653664	best: 41.2653664 (9)	total: 1.47s	remaining: 4m 51s
10:	learn: 40.1959855	test: 41.0617759	best: 41.0617759 (10)	total: 1.61s	remaining: 4m 51s
11:	learn

91:	learn: 37.1521135	test: 38.9890470	best: 38.9890470 (91)	total: 15.5s	remaining: 5m 20s
92:	learn: 37.1329522	test: 38.9722629	best: 38.9722629 (92)	total: 15.6s	remaining: 5m 20s
93:	learn: 37.1173915	test: 38.9636363	best: 38.9636363 (93)	total: 15.8s	remaining: 5m 20s
94:	learn: 37.1113160	test: 38.9643825	best: 38.9636363 (93)	total: 16s	remaining: 5m 20s
95:	learn: 37.0976665	test: 38.9629433	best: 38.9629433 (95)	total: 16.1s	remaining: 5m 20s
96:	learn: 37.0852877	test: 38.9634874	best: 38.9629433 (95)	total: 16.3s	remaining: 5m 20s
97:	learn: 37.0512399	test: 38.9459913	best: 38.9459913 (97)	total: 16.5s	remaining: 5m 20s
98:	learn: 37.0313772	test: 38.9421734	best: 38.9421734 (98)	total: 16.7s	remaining: 5m 19s
99:	learn: 37.0187939	test: 38.9396978	best: 38.9396978 (99)	total: 16.9s	remaining: 5m 20s
100:	learn: 37.0015513	test: 38.9350337	best: 38.9350337 (100)	total: 17s	remaining: 5m 20s
101:	learn: 36.9868382	test: 38.9316634	best: 38.9316634 (101)	total: 17.2s	remain

179:	learn: 36.1586777	test: 38.7143101	best: 38.7143101 (179)	total: 31.5s	remaining: 5m 18s
180:	learn: 36.1480316	test: 38.7156713	best: 38.7143101 (179)	total: 31.7s	remaining: 5m 18s
181:	learn: 36.1385277	test: 38.7155107	best: 38.7143101 (179)	total: 31.8s	remaining: 5m 18s
182:	learn: 36.1356798	test: 38.7150703	best: 38.7143101 (179)	total: 32s	remaining: 5m 17s
183:	learn: 36.1306373	test: 38.7128050	best: 38.7128050 (183)	total: 32.2s	remaining: 5m 17s
184:	learn: 36.1159383	test: 38.7047372	best: 38.7047372 (184)	total: 32.4s	remaining: 5m 17s
185:	learn: 36.1054633	test: 38.7048382	best: 38.7047372 (184)	total: 32.6s	remaining: 5m 17s
186:	learn: 36.0968947	test: 38.7046622	best: 38.7046622 (186)	total: 32.8s	remaining: 5m 17s
187:	learn: 36.0869950	test: 38.7090008	best: 38.7046622 (186)	total: 33s	remaining: 5m 17s
188:	learn: 36.0808281	test: 38.7107829	best: 38.7046622 (186)	total: 33.1s	remaining: 5m 17s
189:	learn: 36.0735193	test: 38.7168414	best: 38.7046622 (186)	t

267:	learn: 35.5017266	test: 38.5566157	best: 38.5566157 (267)	total: 47.3s	remaining: 5m 5s
268:	learn: 35.4938231	test: 38.5477241	best: 38.5477241 (268)	total: 47.5s	remaining: 5m 5s
269:	learn: 35.4931357	test: 38.5472924	best: 38.5472924 (269)	total: 47.6s	remaining: 5m 5s
270:	learn: 35.4896905	test: 38.5459114	best: 38.5459114 (270)	total: 47.8s	remaining: 5m 4s
271:	learn: 35.4825019	test: 38.5414562	best: 38.5414562 (271)	total: 48s	remaining: 5m 4s
272:	learn: 35.4778021	test: 38.5403147	best: 38.5403147 (272)	total: 48.1s	remaining: 5m 4s
273:	learn: 35.4629528	test: 38.5302421	best: 38.5302421 (273)	total: 48.3s	remaining: 5m 4s
274:	learn: 35.4600323	test: 38.5300746	best: 38.5300746 (274)	total: 48.5s	remaining: 5m 4s
275:	learn: 35.4558807	test: 38.5289250	best: 38.5289250 (275)	total: 48.7s	remaining: 5m 4s
276:	learn: 35.4481440	test: 38.5245985	best: 38.5245985 (276)	total: 48.9s	remaining: 5m 3s
277:	learn: 35.4452407	test: 38.5244848	best: 38.5244848 (277)	total: 49

356:	learn: 35.0658129	test: 38.4516904	best: 38.4516904 (356)	total: 1m 3s	remaining: 4m 50s
357:	learn: 35.0626364	test: 38.4507557	best: 38.4507557 (357)	total: 1m 3s	remaining: 4m 50s
358:	learn: 35.0541085	test: 38.4468947	best: 38.4468947 (358)	total: 1m 3s	remaining: 4m 50s
359:	learn: 35.0480792	test: 38.4472089	best: 38.4468947 (358)	total: 1m 3s	remaining: 4m 50s
360:	learn: 35.0449940	test: 38.4498902	best: 38.4468947 (358)	total: 1m 3s	remaining: 4m 50s
361:	learn: 35.0393315	test: 38.4452821	best: 38.4452821 (361)	total: 1m 4s	remaining: 4m 49s
362:	learn: 35.0361390	test: 38.4457026	best: 38.4452821 (361)	total: 1m 4s	remaining: 4m 49s
363:	learn: 35.0324392	test: 38.4453330	best: 38.4452821 (361)	total: 1m 4s	remaining: 4m 49s
364:	learn: 35.0287576	test: 38.4454546	best: 38.4452821 (361)	total: 1m 4s	remaining: 4m 49s
365:	learn: 35.0253495	test: 38.4454023	best: 38.4452821 (361)	total: 1m 4s	remaining: 4m 49s
366:	learn: 35.0222386	test: 38.4429070	best: 38.4429070 (36

...

1570:	learn: 32.3301010	test: 38.2365985	best: 38.2365985 (1570)	total: 4m 41s	remaining: 1m 16s
1571:	learn: 32.3291889	test: 38.2361092	best: 38.2361092 (1571)	total: 4m 41s	remaining: 1m 16s
1572:	learn: 32.3272254	test: 38.2354696	best: 38.2354696 (1572)	total: 4m 41s	remaining: 1m 16s
1573:	learn: 32.3261080	test: 38.2354556	best: 38.2354556 (1573)	total: 4m 42s	remaining: 1m 16s
1574:	learn: 32.3251939	test: 38.2351351	best: 38.2351351 (1574)	total: 4m 42s	remaining: 1m 16s
1575:	learn: 32.3240527	test: 38.2353983	best: 38.2351351 (1574)	total: 4m 42s	remaining: 1m 15s
1576:	learn: 32.3217975	test: 38.2375556	best: 38.2351351 (1574)	total: 4m 42s	remaining: 1m 15s
1577:	learn: 32.3197767	test: 38.2383268	best: 38.2351351 (1574)	total: 4m 42s	remaining: 1m 15s
1578:	learn: 32.3192766	test: 38.2381254	best: 38.2351351 (1574)	total: 4m 42s	remaining: 1m 15s
1579:	learn: 32.3174682	test: 38.2383562	best: 38.2351351 (1574)	total: 4m 43s	remaining: 1m 15s
1580:	learn: 32.3169002	test: 

1655:	learn: 32.1941428	test: 38.2227436	best: 38.2216948 (1654)	total: 4m 56s	remaining: 1m 1s
1656:	learn: 32.1938706	test: 38.2227684	best: 38.2216948 (1654)	total: 4m 56s	remaining: 1m 1s
1657:	learn: 32.1924231	test: 38.2230860	best: 38.2216948 (1654)	total: 4m 57s	remaining: 1m 1s
1658:	learn: 32.1883007	test: 38.2236328	best: 38.2216948 (1654)	total: 4m 57s	remaining: 1m 1s
1659:	learn: 32.1874711	test: 38.2233695	best: 38.2216948 (1654)	total: 4m 57s	remaining: 1m
1660:	learn: 32.1847887	test: 38.2229884	best: 38.2216948 (1654)	total: 4m 57s	remaining: 1m
1661:	learn: 32.1843030	test: 38.2228908	best: 38.2216948 (1654)	total: 4m 57s	remaining: 1m
1662:	learn: 32.1828808	test: 38.2226475	best: 38.2216948 (1654)	total: 4m 58s	remaining: 1m
1663:	learn: 32.1822165	test: 38.2227049	best: 38.2216948 (1654)	total: 4m 58s	remaining: 1m
1664:	learn: 32.1811769	test: 38.2230627	best: 38.2216948 (1654)	total: 4m 58s	remaining: 1m
1665:	learn: 32.1790535	test: 38.2235863	best: 38.2216948 

1742:	learn: 32.0545491	test: 38.2352109	best: 38.2216948 (1654)	total: 5m 12s	remaining: 46.1s
1743:	learn: 32.0526371	test: 38.2335105	best: 38.2216948 (1654)	total: 5m 13s	remaining: 45.9s
1744:	learn: 32.0489421	test: 38.2335306	best: 38.2216948 (1654)	total: 5m 13s	remaining: 45.8s
1745:	learn: 32.0480755	test: 38.2334717	best: 38.2216948 (1654)	total: 5m 13s	remaining: 45.6s
1746:	learn: 32.0468080	test: 38.2329312	best: 38.2216948 (1654)	total: 5m 13s	remaining: 45.4s
1747:	learn: 32.0453131	test: 38.2338156	best: 38.2216948 (1654)	total: 5m 13s	remaining: 45.2s
1748:	learn: 32.0443485	test: 38.2338032	best: 38.2216948 (1654)	total: 5m 13s	remaining: 45.1s
1749:	learn: 32.0426318	test: 38.2336529	best: 38.2216948 (1654)	total: 5m 14s	remaining: 44.9s
1750:	learn: 32.0413345	test: 38.2332859	best: 38.2216948 (1654)	total: 5m 14s	remaining: 44.7s
1751:	learn: 32.0404308	test: 38.2326184	best: 38.2216948 (1654)	total: 5m 14s	remaining: 44.5s
1752:	learn: 32.0363665	test: 38.2324325

1828:	learn: 31.9374196	test: 38.2331961	best: 38.2216948 (1654)	total: 5m 28s	remaining: 30.7s
1829:	learn: 31.9356749	test: 38.2334996	best: 38.2216948 (1654)	total: 5m 28s	remaining: 30.5s
1830:	learn: 31.9347033	test: 38.2333138	best: 38.2216948 (1654)	total: 5m 28s	remaining: 30.3s
1831:	learn: 31.9329547	test: 38.2336669	best: 38.2216948 (1654)	total: 5m 28s	remaining: 30.2s
1832:	learn: 31.9321295	test: 38.2335368	best: 38.2216948 (1654)	total: 5m 28s	remaining: 30s
1833:	learn: 31.9306942	test: 38.2335817	best: 38.2216948 (1654)	total: 5m 29s	remaining: 29.8s
1834:	learn: 31.9302936	test: 38.2336065	best: 38.2216948 (1654)	total: 5m 29s	remaining: 29.6s
1835:	learn: 31.9293942	test: 38.2335306	best: 38.2216948 (1654)	total: 5m 29s	remaining: 29.4s
1836:	learn: 31.9282072	test: 38.2332115	best: 38.2216948 (1654)	total: 5m 29s	remaining: 29.3s
1837:	learn: 31.9261690	test: 38.2331806	best: 38.2216948 (1654)	total: 5m 29s	remaining: 29.1s
1838:	learn: 31.9250728	test: 38.2338837	b

1915:	learn: 31.8103427	test: 38.2320964	best: 38.2216948 (1654)	total: 5m 43s	remaining: 15.1s
1916:	learn: 31.8098531	test: 38.2320546	best: 38.2216948 (1654)	total: 5m 43s	remaining: 14.9s
1917:	learn: 31.8095813	test: 38.2320716	best: 38.2216948 (1654)	total: 5m 43s	remaining: 14.7s
1918:	learn: 31.8077048	test: 38.2329049	best: 38.2216948 (1654)	total: 5m 44s	remaining: 14.5s
1919:	learn: 31.8070681	test: 38.2327841	best: 38.2216948 (1654)	total: 5m 44s	remaining: 14.3s
1920:	learn: 31.8062620	test: 38.2330133	best: 38.2216948 (1654)	total: 5m 44s	remaining: 14.2s
1921:	learn: 31.8053088	test: 38.2327640	best: 38.2216948 (1654)	total: 5m 44s	remaining: 14s
1922:	learn: 31.8039013	test: 38.2330768	best: 38.2216948 (1654)	total: 5m 44s	remaining: 13.8s
1923:	learn: 31.8034340	test: 38.2331124	best: 38.2216948 (1654)	total: 5m 45s	remaining: 13.6s
1924:	learn: 31.8016502	test: 38.2337753	best: 38.2216948 (1654)	total: 5m 45s	remaining: 13.4s
1925:	learn: 31.8010432	test: 38.2338930	b

In [ ]:
prediction = cbr.predict(X[idx_te])
np.sqrt(mean_squared_error(y[idx_te], prediction))

In [ ]:
prediction = cbr.predict(X[idx_tr])
np.sqrt(mean_squared_error(y[idx_tr], prediction))

In [216]:
cat_train.iloc[:, list(set(np.arange(cat_train.shape[1])) - set(caticat))]

,Время отправления по расписанию_year,Время отправления по расписанию_month,Время отправления по расписанию_day,Время отправления по расписанию_hour,Время отправления по расписанию_minute,Время отправления по расписанию_epoch,Время прибытия по расписанию_year,Время прибытия по расписанию_month,Время прибытия по расписанию_day,Время прибытия по расписанию_hour,Время прибытия по расписанию_minute,Время прибытия по расписанию_epoch,arrival_diff,latitude,longitude,num_gates,num_runways,num_terminals,runway_len,size_pax,latitude_arrival,longitude_arrival,num_gates_arrival,num_runways_arrival,num_terminals_arrival,runway_len_arrival,size_pax_arrival
0,2015,10,27,7,40,1445931600,2015,10,27,20,45,1445978700,47100,55.971667,-37.415000,28,2,1,3625,29256,23.016667,82.383056,17,1,1,4000,4119
1,2015,10,27,9,50,1445939400,2015,10,27,20,35,1445978100,38700,55.971667,-37.415000,28,2,1,3625,29256,40.639722,73.778889,137,4,7,3372,50423
2,2015,10,27,10,45,1445942700,2015,10,27,23,35,1445988900,46200,55.971667,-37.415000,28,2,1,3625,29256,25.793056,80.290556,120,4,3,3159,40500
3,2015,10,27,12,30,1445949000,2015,10,28,1,20,1445995200,46200,55.971667,-37.415000,28,2,1,3625,29256,33.942500,118.408056,112,4,9,3230,66667
4,2015,10,27,14,15,1445955300,2015,10,27,16,40,1445964000,8700,44.573611,-26.103333,24,2,1,3500,7643,55.971667,-37.415000,28,2,1,3625,29256
5,2015,10,27,14,30,1445956200,2015,10,27,17,15,1445966100,9900,53.630278,-9.988056,25,2,1,3458,13558,55.971667,-37.415000,28,2,1,3625,29256
6,2015,10,27,14,35,1445956500,2015,10,28,1,25,1445995500,39000,55.971667,-37.415000,28,2,1,3625,29256,40.639722,73.778889,137,4,7,3372,50423
7,2015,10,27,15,40,1445960400,2015,10,27,21,20,1445980800,20400,25.254722,-55.364167,101,2,2,4223,66431,55.971667,-37.415000,28,2,1,3625,29256
8,2015,10,27,16,10,1445962200,2015,10,28,0,35,1445992500,30300,55.971667,-37.415000,28,2,1,3625,29256,43.399167,-132.151389,16,4,1,1893,1853
9,2015,10,27,16,45,1445964300,2015,10,27,20,55,1445979300,15000,32.011389,-34.886667,30,3,1,2984,12978,55.971667,-37.415000,28,2,1,3625,29256


In [183]:
dict(zip(np.arange(0, cat_train.shape[0]), cat_train.columns))

{0: 'Рейс',
 1: 'А/П отправл',
 2: 'А/П прибыт',
 3: 'Номер ВС',
 4: 'Время отправления по расписанию_year',
 5: 'Время отправления по расписанию_month',
 6: 'Время отправления по расписанию_day',
 7: 'Время отправления по расписанию_hour',
 8: 'Время отправления по расписанию_minute',
 9: 'Время отправления по расписанию_dow',
 10: 'Время отправления по расписанию_epoch',
 11: 'Время прибытия по расписанию_year',
 12: 'Время прибытия по расписанию_month',
 13: 'Время прибытия по расписанию_day',
 14: 'Время прибытия по расписанию_hour',
 15: 'Время прибытия по расписанию_minute',
 16: 'Время прибытия по расписанию_dow',
 17: 'Время прибытия по расписанию_epoch',
 18: 'arrival_diff',
 19: 'country',
 20: 'latitude',
 21: 'longitude',
 22: 'num_gates',
 23: 'num_runways',
 24: 'num_terminals',
 25: 'runway_len',
 26: 'runway_material',
 27: 'season',
 28: 'size_pax',
 29: 'size_value',
 30: 'town',
 31: 'type',
 32: 'country_arrival',
 33: 'latitude_arrival',
 34: 'longitude_arrival',
 

In [195]:
caticat = icats.copy()
caticat.extend([0, 3, 9, 16, 45, 46, 47, 48])
# cat_train.iloc[:, icats]

## Random forest

In [159]:
np.argsort(-cbr.feature_importances_)

array([45,  0, 10, 14, 21, 17, 18,  3,  7, 12, 46,  6,  5, 20, 13, 28, 25,
       47, 48,  9, 34,  1, 15, 22, 41, 26, 31, 38, 33,  8, 16, 35, 29, 19,
       43, 23, 24, 30, 44, 32, 42, 39, 11, 27, 40,  4, 37,  2, 36])

In [165]:
cats = [np.issubdtype(dtt, np.number) for dtt in cat_train.dtypes]

In [168]:
le_features = cat_train.copy()

In [169]:
for i in icats:
    le_features.iloc[:, i] = LabelEncoder().fit_transform(le_features.iloc[:, i])

In [174]:
regressor = RandomForestRegressor(n_estimators=500, n_jobs=6, max_depth=7, random_state=42)
regressor.fit(le_features.values[idx_tr], y[idx_tr])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=6,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [175]:
prediction = regressor.predict(le_features.values[idx_te])

In [177]:
np.sqrt(mean_squared_error(y[idx_te], prediction))

39.822412450431145

In [221]:
xgr = xgb.XGBRegressor(max_depth=8, learning_rate=0.2, n_estimators=500, objective='reg:linear', n_jobs=9)
xgr.fit(le_features.values[idx_tr], y[idx_tr])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=9, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [223]:
prediction = xgr.predict(le_features.values[idx_tr])
np.sqrt(mean_squared_error(y[idx_tr], prediction))

22.593850773155964

In [47]:
cat_train[["country_arrival", "country"]] = cat_train[["country_arrival", "country"]].astype(np.int32)

In [73]:
def generate_mean_encoding(column, target):
    column_name = column.name
    return pd.concat([column, target],
                     axis='columns').groupby(column_name).mean()[target.name]


In [50]:
lx = train.shape[0] - test.shape[0]

In [61]:
icats = [i for i, dtt in enumerate(cat_train.dtypes) if not np.issubdtype(dtt, np.number)]
icats.extend([0, 3, 9, 16, 19, 32, 45, 46, 47, 48])
icats = np.sort(icats)

In [199]:
icats

(22,)

In [83]:
encodings_mean = [generate_mean_encoding(cat_train.iloc[:, i], target).fillna(0) for i in icats]

In [77]:
encodings_VC = [cat_train.iloc[:, i].value_counts() for i in icats]

In [85]:
encodings_lce = [LabelCountEncoder().fit_transform(cat_train.iloc[:, i]) for i in icats]

In [86]:
encodings_le = [LabelEncoder().fit_transform(cat_train.iloc[:, i]) for i in icats]

In [92]:
%%time
r = [cat_train.iloc[:, col].apply(lambda x: encodings_VC[i].get(x)) for i, col in enumerate(icats)]

CPU times: user 2min 1s, sys: 200 ms, total: 2min 1s
Wall time: 2min 1s


In [93]:
%%time
z = [cat_train.iloc[:, col].apply(lambda x: encodings_mean[i].get(x)) for i, col in enumerate(icats)]

CPU times: user 1min 36s, sys: 151 ms, total: 1min 36s
Wall time: 1min 36s


In [106]:
data = pd.DataFrame(np.array(encodings_le).T, columns = cat_train.columns[icats])

In [110]:
data = data.join(pd.DataFrame(np.array(encodings_lce).T, columns = cat_train.columns[icats]), rsuffix="_lce")

In [121]:
data = data.join(pd.concat(r, axis=1), rsuffix="_VC", lsuffix="_lc")

In [122]:
data = data.join(pd.concat(z, axis=1), rsuffix="_ME", lsuffix="_lc")

In [130]:
data = data.join(cat_train.iloc[:, list(set(np.arange(cat_train.shape[1])) - set(icats))])

In [137]:
X, y = data.values[:lx], target.values[:lx]

In [138]:
kf = KFold(n_splits=5, shuffle=True, random_state=1234)

In [168]:
regrs = []
predictions = []
for idx_tr, idx_te in kf.split(X):
    regressor = RandomForestRegressor(n_estimators=500, n_jobs=9, max_depth=7, random_state=42, verbose=True)
    regressor.fit(X[idx_tr], y[idx_tr])
    prediction = regressor.predict(X[idx_te])
    print(np.sqrt(mean_squared_error(y[idx_te], prediction)))
    predictions.append(prediction)
    regrs.append(regressor)

[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:  1.4min
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:  7.5min
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed: 17.5min
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed: 20.0min finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.3s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.7s finished


38.67325240387334


[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:  1.4min
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:  7.1min
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed: 16.5min
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed: 19.1min finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.2s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.7s finished


38.10449966876701


[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:  1.4min
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:  7.0min
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed: 16.3min
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed: 18.7min finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.2s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.7s finished


...

[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:  1.3min
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:  6.9min
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed: 16.2min
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed: 18.5min finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.3s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.7s finished


37.34021279239205


[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:  1.4min
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:  6.8min
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed: 16.2min
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed: 18.6min finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.2s


38.50316779747674


[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.7s finished


In [171]:
data.values.shape

(740943, 115)

In [169]:
ans = np.zeros((5, data.values[lx:].shape[0]))
for i, regr in enumerate(regrs):
    pred = regr.predict(data.values[lx:])
    ans[i] = pred

[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.1s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.1s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.1s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.1s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.1s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.1s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.1s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.1s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.

In [172]:
res_df = pd.DataFrame(gmean(ans), columns=["Задержка отправления в минутах"])
res_df.to_csv("./out/gmean_big_rf_wide_all_encoded_features.csv", index=True)

In [173]:
stack_df = []
for i, (idx_tr, idx_te) in enumerate(kf.split(X)):
    stack_df.append(pd.DataFrame(regrs[i].predict(data.values[idx_te]), index=idx_te))

[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.2s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.7s finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.2s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.7s finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.2s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.7s finished
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:    0.0s
[Parallel(n_jobs=9)]: Done 182 tasks      | elapsed:    0.2s
[Parallel(n_jobs=9)]: Done 432 tasks      | elapsed:    0.6s
[Parallel(n_jobs=9)]: Done 500 out of 500 | elapsed:    0.

In [174]:
sss = pd.concat(stack_df)

In [175]:
sss.sort_index().to_csv("./stacking/big_rfgmean_wide.csv", index=True)

In [197]:
# joblib.dump(regrs[0], "./rf_model/rf_0.joblib")
# joblib.dump(regrs[1], "./rf_model/rf_1.joblib")
# joblib.dump(regrs[2], "./rf_model/rf_2.joblib")
# joblib.dump(regrs[3], "./rf_model/rf_3.joblib")
# joblib.dump(regrs[4], "./rf_model/rf_4.joblib")

['./rf_model/rf_4.joblib']

In [198]:
regrs[0]

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=9,
           oob_score=False, random_state=42, verbose=True,
           warm_start=False)

In [ ]:
etree = ET.parse("./data/airports.xml") #create an ElementTree object 
root = etree.getroot()
airport_data = parse_xml(root)
etree = ET.parse("./data/airports_asia.xml") #create an ElementTree object 
root = etree.getroot()
airport_data.update(parse_xml(root))
etree = ET.parse("./data/airports_africa.xml") #create an ElementTree object 
root = etree.getroot()
airport_data.update(parse_xml(root))
etree = ET.parse("./data/airports_south_america.xml") #create an ElementTree object 
root = etree.getroot()
airport_data.update(parse_xml(root))
etree = ET.parse("./data/airports_north_america.xml") #create an ElementTree object 
root = etree.getroot()
airport_data.update(parse_xml(root))

In [17]:
df_airport = pd.DataFrame(airport_data).transpose()

In [18]:
df_airport.loc[df_airport.size_pax.isna(), "size_pax"] = 960
df_airport.size_pax = df_airport.size_pax.astype(np.int32)

In [20]:
df_airport.loc[df_airport.runway_len.isna(), "runway_len"] = 0
df_airport.loc[df_airport.runway_len.isna(), "runway_material"] = "Asphalt"

In [21]:
df_airport.loc[:, ["num_gates", "num_runways", "num_terminals", "runway_len"]] = \
    df_airport[["num_gates", "num_runways", "num_terminals", "runway_len"]].astype(np.int32)

In [22]:
df_airport.loc[df_airport.longitude == "109'57'48''E", "longitude"] = "109°57'48''E"

In [271]:
etree = ET.parse("./data/airports.xml") #create an ElementTree object 
root = etree.getroot()
# offers = root.getchildren()

In [195]:
train[(train["Время прибытия по расписанию"] <= datetime(2016, 10, 28)) & 
      (train["Время отправления по расписанию"] >= datetime(2016, 10, 26)) & 
     (train["А/П отправл"] == 'SVO') & (train["А/П прибыт"] == 'KZN')]   \
        [["Время отправления по расписанию", "Время прибытия по расписанию", "Рейс"]] \
    .sort_values(["Время отправления по расписанию"], axis=0) 
# - 6 часов 

,Время отправления по расписанию,Время прибытия по расписанию,Рейс
215384,2016-10-26 00:30:00,2016-10-26 02:00:00,128
215514,2016-10-26 06:45:00,2016-10-26 08:20:00,193
215558,2016-10-26 08:15:00,2016-10-26 09:45:00,191
215578,2016-10-26 09:10:00,2016-10-26 10:40:00,122
215761,2016-10-26 15:05:00,2016-10-26 16:40:00,124
215854,2016-10-26 18:40:00,2016-10-26 20:15:00,126
216004,2016-10-27 00:30:00,2016-10-27 02:00:00,128
216129,2016-10-27 06:45:00,2016-10-27 08:20:00,193
216274,2016-10-27 08:15:00,2016-10-27 09:45:00,191
216194,2016-10-27 09:10:00,2016-10-27 10:40:00,122


In [196]:
additional_data[(additional_data.SCH_START_DATE_TIME >= datetime(2016, 10, 26)) & 
                (additional_data.SCH_END_DATE_TIME <= datetime(2016, 10, 28)) & 
            (additional_data.DEPARTURE_AIRPORT_CODE == "SVO") & (additional_data.ARRIVAL_AIRPORT_CODE == 'KZN')] \
        [["SCH_START_DATE_TIME", "SCH_END_DATE_TIME", "FLIGHT_NUMBER", "TAIL_NUMBER", "DISPLAY_CODE"]]  \
    .sort_values(["SCH_START_DATE_TIME"], axis=0)

,SCH_START_DATE_TIME,SCH_END_DATE_TIME,FLIGHT_NUMBER,TAIL_NUMBER,DISPLAY_CODE
663241,2016-10-26 00:45:00,2016-10-26 02:20:00,1270,VQBBC,SVOA320SU
666512,2016-10-26 02:15:00,2016-10-26 03:45:00,1268,VQBSL,SVOA32ASU
661878,2016-10-26 03:10:00,2016-10-26 04:40:00,1190,VQBKT,SVOA320SU
665767,2016-10-26 09:05:00,2016-10-26 10:40:00,1192,VQBSE,SVOA32ASU
664056,2016-10-26 12:40:00,2016-10-26 14:15:00,1194,89026,SVOSU95SU
662787,2016-10-26 18:30:00,2016-10-26 20:00:00,1196,89057,SVOSU95SU
663969,2016-10-27 00:45:00,2016-10-27 02:20:00,1270,VPBRX,SVOA320SU
661303,2016-10-27 02:15:00,2016-10-27 03:45:00,1268,89061,SVOSU95SU
662678,2016-10-27 03:10:00,2016-10-27 04:40:00,1190,VQBIV,SVOA320SU
665196,2016-10-27 09:05:00,2016-10-27 10:40:00,1192,VPBJY,SVOA32ASU
